![Xilinx Logo](images/xilinx_logo.png "Xilinx Logo")

# 1. Introduction

This notebook demonstrates how to playback an encoded video file and display it inside this jupyter notebook using the GStreamer multimedia framework.

The example video file used in this notebook is VP9 encoded (https://www.webmproject.org/vp9/) and uses the webm container format (https://www.webmproject.org/docs/container/).

The video pipeline is composed of the following GStreamer elements:
* The ``multifilesrc`` element is used to read the file from disk
* The ``vp9dec`` and ``matroskademux`` elements are used to decode and demux the video file. The file encoding is auto-detected and the required element auto-instantiated by the ``decodebin`` element.
* The ``videoconvert`` and ``jpegenc`` element are used to convert and compress the raw video format to JPEG.
* The ``appsink`` element is used to make the JPEG frames available to the jupyter notebook where they are displayed.

In this notebook, you will:
1. Create a GStreamer video pipeline that decodes a video file and displays the video inside this notebook.
2. Create a GStreamer pipeline graph and view it inside this notebook.

# 2. Imports and Initialization

Import all python modules required for this notebook. 

In [ ]:
from IPython.display import Image, display, clear_output
import pydot
import sys
import gi
gi.require_version('Gst', '1.0')
gi.require_version("GstApp", "1.0")
from gi.repository import GObject, GLib, Gst, GstApp

This is the Base TRD notebook 1 (nb1).

In [ ]:
nb = "nb1"

Create a directory for saving the pipeline graph as dot file. Set the GStreamer debug dot directory environment variable to point to that directory.

In [ ]:
dotdir = "/home/root/gst-dot/" + nb
!mkdir -p $dotdir
%env GST_DEBUG_DUMP_DOT_DIR = $dotdir

Initialize the GStreamer library. Enable debug by setting the debug string, set default to level 1 for all categories.

In [ ]:
Gst.init(None)
Gst.debug_set_threshold_from_string('*:1', True)

# 3. Create and Configure the GStreamer Elements

Create the ``filesrc`` element and set some properties:
* Set the ``location`` property to the path of the video file.
* Set the ``loop`` property to ``True`` to play the video in a continuous loop.

In [ ]:
file_name = "/usr/share/movies/Big_Buck_Bunny_4K.webm.360p.vp9.webm"
loop = True

src = Gst.ElementFactory.make("multifilesrc")
src.set_property("location", file_name)
src.set_property("loop", loop)

The decoder source pad is created dynamically when it detects the incoming stream. The ``pad-added`` signal is emitted and this ``pad_added`` callback function is executed. It links the source pad of the decoder to the sink pad of the ``videoconvert`` element.

In [ ]:
def pad_added(element, pad):
    global cvt
    
    sink_pad = cvt.get_static_pad("sink")
    if not sink_pad.is_linked():
        pad.link(sink_pad)

Create the ``decodebin`` element. The ``decodebin`` instantiates the ``matroskademux`` and ``vp9dec`` elements to demux and decode the sample video file. Currently supported video codecs are theora, vp8 and vp9. Theora video files typically use an ogg container, whereas vp8 and vp9 video files typically use a matroska/webm container.

Register the above ``pad_added`` callback function with the ``pad-added`` signal of the ``decodebin`` element.

In [ ]:
decodebin = Gst.ElementFactory.make("decodebin")
decodebin.connect("pad-added", pad_added);

Create the ``videoconvert`` and ``jpegenc`` elements.

The ``videoconvert`` element converts the I420 input format to YUY2.

The ``jpegenc`` element compresses the YUY2 frame to JPEG.

In [ ]:
cvt = Gst.ElementFactory.make("videoconvert")
jpegenc = Gst.ElementFactory.make("jpegenc")

Create a callback function ``new_sample`` that retrieves the JPEG data from a GStreamer buffer object and passes it to the ``display`` function of the ``IPython.display`` module which displays the video frame inside the notebook.

In [ ]:
def new_sample(sink):  
    sample = sink.pull_sample()
    buffer = sample.get_buffer()
    ret, info = buffer.map(Gst.MapFlags.READ)
    
    display(Image(data=info.data))
    clear_output(wait=True)
    
    buffer.unmap(info)
    
    return Gst.FlowReturn.OK

Create the ``appsink`` element and set some properties:
* Set the ``drop`` property to ``True`` to drop old buffers when the buffer queue is full
* Set the ``max-buffers`` property to 0 to queue an unlimited number of buffers
* Set the ``emit-signals`` property to ``True`` to emit the ``new-sample`` signal

Register the above ``new_sample`` callback function with the ``new-sample`` signal of the ``appsink`` element.

In [ ]:
sink = Gst.ElementFactory.make("appsink")
sink.set_property("drop", True)
sink.set_property("max_buffers", 0)
sink.set_property("emit-signals", True)
sink.connect("new-sample", new_sample);

# Uncomment the below code to read back the newly set property values
#print("appsink properties: ")
#print("drop: " + str(sink.get_property("drop")))
#print("max_buffers: " + str(sink.get_property("max_buffers")))
#print("emit-signals: " + str(sink.get_property("emit-signals")))

# 4. Create and Run the GStreamer Pipeline

Create the pipeline, add all elements, and link them together.

**Note:** The ``decodebin`` element is not linked to the ``videoconvert`` element at this point. The link is created dynamically in the ``pad_added`` callback function once the ``decodebin`` pad has been added.

In [ ]:
pipeline = Gst.Pipeline.new(nb)

pipeline.add(src)
pipeline.add(decodebin)
pipeline.add(cvt)
pipeline.add(jpegenc)
pipeline.add(sink)

src.link(decodebin)
cvt.link(jpegenc)
jpegenc.link(sink);

The ``bus_call`` function listens on the bus for ``EOS`` and ``ERROR`` events. If any of these events occur, stop the pipeline (set to ``NULL`` state) and quit the main loop.

In case of an ``ERROR`` event, parse and print the error message.

In [ ]:
def bus_call(bus, message, loop):
    t = message.type
    if t == Gst.MessageType.EOS:
        sys.stdout.write("End-of-stream\n")
        pipeline.set_state(Gst.State.NULL)
        loop.quit()
    elif t == Gst.MessageType.ERROR:
        err, debug = message.parse_error()
        sys.stderr.write("Error: %s: %s\n" % (err, debug))
        pipeline.set_state(Gst.State.NULL)
        loop.quit()
    return True

Start the pipeline (set to ``PLAYING`` state), create the main loop and listen to messages on the bus. Register the ``bus_call`` callback function with the ``message`` signal of the bus. Start the main loop.

The video frames will be displayed below the following code cell. 

To stop the pipeline, click the square shaped icon labelled 'Interrupt the kernel' in the top menu bar. Create a dot graph of the pipeline topology before stopping the pipeline. Quit the main loop.

In [ ]:
pipeline.set_state(Gst.State.PLAYING);

loop = GLib.MainLoop()
bus = pipeline.get_bus()
bus.add_signal_watch()
bus.connect("message", bus_call, loop)

try:
    loop.run()
except:
    sys.stdout.write("Interrupt caught\n")
    Gst.debug_bin_to_dot_file(pipeline, Gst.DebugGraphDetails.ALL, nb)
    pipeline.set_state(Gst.State.NULL)
    loop.quit()
    pass

# 5. View the GStreamer Pipeline Graph

Convert the dot file to png and display the pipeline graph. The image will be displayed below the following code cell. Double click on the generate image file to zoom in.

**Note:** This step may take a few seconds.

In [ ]:
dotfile = dotdir + "/" + nb + ".dot"
graph = pydot.graph_from_dot_file(dotfile, 'utf-8')
display(Image(graph[0].create(None, 'png', 'utf-8')))

# 6. Summary

In this notebook you learned how to:
1. Create a GStreamer pipeline that demonstrates how to decode a video file and play it back inside the jupyter notebook
2. Export the pipeline topology as a dot file image and display it in the notebook

<center>Copyright© 2019 Xilinx</center>